Method 1
=========
- Compute the mean of each seen class.

- Compute the similarity (dot product based) of each unseen class with each of the seen classes.

- Normalize the similarity vector (to that it sums to 1, since we are using a convex combination).

- Compute the mean of each unseen class using a convex combination of means of seen classes.

- Apply the model to predict labels on unseen class test inputs.

- Compute classification accuracies.

Brief description of Dataset -->

X_seen=np.load('X_seen.npy') 	(40 x N_i x D): 40 feature matrices. X_seen[i] is the N_i x D feature matrix of seen class i

Xtest=np.load('Xtest.npy')	(6180, 4096): feature matrix of the test data.

Ytest=np.load('Ytest.npy',)	(6180, 1): ground truth labels of the test data

class_attributes_seen=np.load('class_attributes_seen.npy')	(40, 85): 40x85 matrix with each row being the 85-dimensional class attribute vector of a seen class.

class_attributes_unseen=np.load('class_attributes_unseen.npy')	(10, 85): 10x85 matrix with each row being the 85-dimensional class attribute vector of an  unseen class.




In [21]:
import numpy as np

In [22]:
""" Loading provided datasets"""

X_seen=np.load('X_seen.npy',allow_pickle=True,encoding='bytes')
Xtest=np.load('Xtest.npy',allow_pickle=True,encoding='bytes')
Ytest=np.load('Ytest.npy',allow_pickle=True,encoding='bytes')
class_attributes_seen=np.load('class_attributes_seen.npy',allow_pickle=True,encoding='bytes')
class_attributes_unseen=np.load('class_attributes_unseen.npy',allow_pickle=True,encoding='bytes')

In [23]:
""" calculating mean (i.e. prototype vector) for seen classes"""

mean_of_seen_cls = []
for cls in X_seen:
    summation = np.zeros(4096,)
    for each_img in cls:
        summation = summation+each_img
    mean_of_seen_cls.append(summation/len(cls))
    
# print(len(mean))
# print(len((mean_of_seen_cls[0])))

In [24]:
# count = 0
# np.shape(np.transpose(class_attributes_unseen[count]))
# # np.shape(mean_of_seen_cls[])
# # class_attributes_seen[10]

In [25]:
""" calculating mean of unseen classes (prototypr vectors for unseen classes) by calculating similarity matrix"""
mean_of_unseen_cls = []
count = 0
while(count<10):
    similarity = []
    for k in range(40):
        similarity.append(np.transpose(class_attributes_unseen[count]) @ class_attributes_seen[k])
    summation = sum(similarity)
    similarity = np.array(similarity)/summation
    sum_arr = np.zeros(4096,)
    for k in range(40):
        sum_arr = sum_arr + similarity[k]*np.array(mean_of_seen_cls[k])
    mean_of_unseen_cls.append(sum_arr)
    count=count+1
    
# len(mean_of_unseen_cls[0])

In [26]:
""" Predicting labels for each test image using calculated prototype vectors for unseen classes"""

Ypred = []
for sample in Xtest:
    count=0
    minimum = 10**100   
    """initializing a large value to minimum"""
    while(count<10):
        L2_norm = np.linalg.norm(sample-np.array(mean_of_unseen_cls[count]))
        """if current norm(distance) is minimum then hold this for predicting label"""
        if(L2_norm < minimum): 
            minimum = L2_norm
            pred_class = count+1
        count = count+1
    Ypred.append(pred_class)

# Ypred

In [27]:
""" taking transpose of predicted list by making this list an array"""
Ypred = np.transpose(np.array(Ypred))
Ypred = Ypred[:,np.newaxis]

# print(np.shape(Ypred))
# print(np.shape(Ytest))

In [28]:
""" defining a function for accuracy :
(accuracy is the percentage of test inputs with correctly predicted classes)"""

def accuracy(arr1,arr2):
    """ this function takes 2 lists as input and calculates accuracy by maching corresponding values."""
    count = 0
    for value in range(len(arr1)):
        if(arr1[value]==arr2[value]):
            count = count+1
    return count*100/len(arr1)

"""calculating accuracy of prediction by passing actual labels and predicted labels"""
print("The accuracy for method 1 is:",accuracy(Ypred,Ytest))

The accuracy for method 1 is: 46.89320388349515
